In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import datetime

In [2]:
# 下載股票資料
df = yf.download('AAPL', start='2020-01-01', end='2023-01-01')

# 清理掉多重索引
df.columns = df.columns.droplevel(1)
df

/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_29586/534702881.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2020-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,72.468269,72.528589,71.223267,71.476607,135480400
2020-01-03,71.763748,72.523777,71.539360,71.696190,146322800
2020-01-06,72.335556,72.374162,70.634539,70.885472,118387200
2020-01-07,71.995361,72.600968,71.775796,72.345212,108872000
2020-01-08,73.153488,73.455087,71.698574,71.698574,132079200
...,...,...,...,...,...
2022-12-23,129.900299,130.451974,127.713291,128.974267,63814900
2022-12-27,128.097458,129.456953,126.806930,129.427400,69007800
2022-12-28,124.166786,129.082622,123.999315,127.742834,85438400


In [3]:
# 改成 bt 可接受的格式
data = bt.feeds.PandasData(dataname=df)

In [4]:
# 設定回測執行的大腦
cerebro = bt.Cerebro()

# 將資料加到回測大腦中
cerebro.adddata(data)

# 設定初始資金
cerebro.broker.setcash(100000)

# 設定交易手續費
cerebro.broker.setcommission(commission=0.001) 

In [5]:
import backtrader as bt

class MyMaCrossStrategy(bt.Strategy):
    params = (
        ('fast_period', 10),
        ('slow_period', 30),
    )

    def __init__(self):
        # 收盤價
        self.dataclose = self.datas[0].close
        
        # 移動平均線
        self.fast_ma = bt.indicators.SMA(self.dataclose, period=self.p.fast_period)
        self.slow_ma = bt.indicators.SMA(self.dataclose, period=self.p.slow_period)

        # 上一根是否為金叉，方便偵測交叉
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} - {txt}')
    
    def next(self):
        # 印出 debug 資訊
        self.log(f"Fast: {self.fast_ma[0]:.2f}, Slow: {self.slow_ma[0]:.2f}")

        # 金叉（快線上穿慢線）
        if self.crossover > 0:
            if not self.position:  # 沒持倉才買
                self.buy()
                self.log(f"BUY EXECUTED: Close={self.dataclose[0]:.2f}")

        # 死叉（快線下穿慢線）
        elif self.crossover < 0:
            if self.position:  # 有持倉才賣
                self.sell()
                self.log(f"SELL EXECUTED: Close={self.dataclose[0]:.2f}")


In [6]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
# 注意：為了讓程式碼可執行，這裡只是展示 Indicator 的邏輯，
# 實際執行需要設置 Cerebro、Data Feed 和 Strategy。

# --- 1. 定義策略 ---
class RsiSmaStrategy(bt.Strategy):
    """
    展示如何計算 RSI 的移動平均線
    """
    params = (
        ('rsi_period', 14),   # RSI 的週期
        ('sma_period', 9),    # 應用於 RSI 上的 SMA 週期
    )

    def __init__(self):
        # 取得第一個數據源 (self.data)
        data = self.datas[0]

        # --- 第一步：計算 RSI ---
        # 這裡會創建一個新的指標實例，它的輸出線就是 RSI 的值
        self.rsi = bt.indicators.RSI(
            data, 
            period=self.p.rsi_period
        )

        # --- 第二步：應用 SMA 到 RSI 的輸出線上 (核心的 Chaining/Composability) ---
        # 
        # rsi (指標實例) 被當作輸入 (data) 傳給 SMA
        # 
        self.rsi_sma = bt.indicators.SimpleMovingAverage(
            self.rsi,   # <--- 這裡！將 RSI 的輸出線作為 SMA 的輸入
            period=self.p.sma_period
        )
        
        # 示範一個訊號：當原始 RSI 向上突破其 SMA 時
        self.crossover = bt.indicators.CrossUp(self.rsi, self.rsi_sma)
        
        # 也可以直接存取這些線：
        # print('當前 RSI:', self.rsi[0])
        # print('當前 RSI 的 SMA:', self.rsi_sma[0])
        
    def next(self):
        # 範例：如果 RSI 向上突破 RSI-SMA，則考慮買入
        if self.crossover[0]:
            # 這裡可以放置買入邏輯，例如 self.buy()
            print(f'{self.datetime.date(0)} - 原始 RSI 突破 SMA，考慮買入！')
            pass

# --- 2. 核心概念解釋 ---

# 如果我們按照傳統方式計算 SMA，我們通常會這樣寫：
# regular_sma = bt.indicators.SimpleMovingAverage(data.close, period=20) 
# -> SMA 應用於**收盤價**

# 但在範例中，我們這樣寫：
# self.rsi_sma = bt.indicators.SimpleMovingAverage(self.rsi, period=self.p.sma_period)
# -> SMA 應用於 **self.rsi** 這個指標實例（即 **RSI 的輸出線**）

# backtrader 的指標系統自動處理了這個「鏈接」的過程！

In [7]:
# 執行策略
print("\nRunning backtest...")

print(f"Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}")

# 加入策略
cerebro.addstrategy(RsiSmaStrategy)
# Run the backtest
results = cerebro.run()

# Get the strategy instance
first_strategy = results[0]

print(f"Final Portfolio Value: {cerebro.broker.getvalue():,.2f}")
print("Backtest complete.")


Running backtest...
Starting Portfolio Value: 100,000.00
2020-02-06 - 原始 RSI 突破 SMA，考慮買入！
2020-02-12 - 原始 RSI 突破 SMA，考慮買入！
2020-03-02 - 原始 RSI 突破 SMA，考慮買入！
2020-03-10 - 原始 RSI 突破 SMA，考慮買入！
2020-03-13 - 原始 RSI 突破 SMA，考慮買入！
2020-03-24 - 原始 RSI 突破 SMA，考慮買入！
2020-04-02 - 原始 RSI 突破 SMA，考慮買入！
2020-04-06 - 原始 RSI 突破 SMA，考慮買入！
2020-04-24 - 原始 RSI 突破 SMA，考慮買入！
2020-04-29 - 原始 RSI 突破 SMA，考慮買入！
2020-05-18 - 原始 RSI 突破 SMA，考慮買入！
2020-05-20 - 原始 RSI 突破 SMA，考慮買入！
2020-05-22 - 原始 RSI 突破 SMA，考慮買入！
2020-06-01 - 原始 RSI 突破 SMA，考慮買入！
2020-06-05 - 原始 RSI 突破 SMA，考慮買入！
2020-06-16 - 原始 RSI 突破 SMA，考慮買入！
2020-06-22 - 原始 RSI 突破 SMA，考慮買入！
2020-06-25 - 原始 RSI 突破 SMA，考慮買入！
2020-07-06 - 原始 RSI 突破 SMA，考慮買入！
2020-07-20 - 原始 RSI 突破 SMA，考慮買入！
2020-07-31 - 原始 RSI 突破 SMA，考慮買入！
2020-08-19 - 原始 RSI 突破 SMA，考慮買入！
2020-08-31 - 原始 RSI 突破 SMA，考慮買入！
2020-09-22 - 原始 RSI 突破 SMA，考慮買入！
2020-09-25 - 原始 RSI 突破 SMA，考慮買入！
2020-10-07 - 原始 RSI 突破 SMA，考慮買入！
2020-10-09 - 原始 RSI 突破 SMA，考慮買入！
2020-10-29 - 原始 RSI 突破 SMA，考慮買入！
2020-11-04 - 原始 RS

In [8]:
# 透過 matplotlib 繪製圖表
import matplotlib.pyplot as plt

print("\nGenerating plot...")

try:
    cerebro.plot(
        style='candlestick',
        barup='red',
        bardown='green',
        volume=True,
        iplot=False,   # 背後使用 matplotlib，而非 Bokeh
        show=False
    )
    print("Plot displayed.")

except Exception as e:
    print(f"Could not plot results: {e}")
    print("Make sure matplotlib is installed and working correctly.")


Generating plot...
Plot displayed.
